In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, classification_report
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import recall_score, precision_score, f1_score
from sklearn.metrics import classification_report

In [ ]:
X = df.drop(['Class', 'Time'], axis=1)
y = df['Class']

In [ ]:
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [ ]:
k_values = range(1, 21)
recalls = []
precisions = []
f1_scores = []

print(f"\n{'k':<5} {'Recall':<12} {'Precision':<12} {'F1-Score':<12}")
print("-" * 45)

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    y_val_pred = knn.predict(X_val_scaled)


    recall = recall_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred)
    f1 = f1_score(y_val, y_val_pred)

    recalls.append(recall)
    precisions.append(precision)
    f1_scores.append(f1)

    print(f"{k:<5} {recall:<12.4f} {precision:<12.4f} {f1:<12.4f}")

In [ ]:
best_k_idx = np.argmax(recalls)
best_k = k_values[best_k_idx]
best_recall = recalls[best_k_idx]

In [ ]:
print(f"\nmelhor k: {best_k} (recall)")
print(f"recall: {best_recall:.4f}")
print(f"precisicao: {precisions[best_k_idx]:.4f}")

In [ ]:
X_train_df = pd.DataFrame(X_train_scaled)
X_val_df = pd.DataFrame(X_val_scaled)

In [ ]:
X_final_train = pd.concat([X_train_df, X_val_df])
y_final_train = pd.concat([y_train, y_val])

In [ ]:
# decidi usar o recall porque ele se adequa melhor no cenário de fraudes
best_k = 1
final_knn = KNeighborsClassifier(n_neighbors=best_k, n_jobs=-1)
final_knn.fit(X_final_train, y_final_train)

In [ ]:
#testando
y_test_pred = final_knn.predict(X_test_scaled)

In [ ]:
# achei o resultado bem interessante, pq ele teve 0.88, e 0.79 identificacoes de fraudes

print("resultado")
print(classification_report(y_test, y_test_pred))